# Spelling correction using LSTMs

This is my undergraduate final project.
We'll try and train a RNN for spelling correction of search queries.

## 1. Importing relevant data and libraries.

In this section we will import the necessary libraries and the datasets and preprocess them for the model to train on.

### 1.1. Importing the libraries

In [1]:
import pandas as pd
import numpy as np
from utils import preprocess_data, string_to_int

### 1.2. Importing and preprocessing the datasets

#### 1.2.1. faspell
FASpell dataset was developed for the evaluation of spell checking algorithms. It contains a set of pairs of misspelled Persian words and their corresponding corrected forms similar to the ASpell dataset used for English.

In [2]:
# import faspell_main
data_faspell = pd.read_csv('data/faspell_main.txt', sep='\t')
data_faspell.head()

,#misspelt,corrected,error-category
0,آاهي,آگاهي,1
1,آبات,آیات,1
2,آبباشد,آب باشد,2
3,آبد,آید,1
4,آبری,عابری,0


In [3]:
data_faspell.drop('error-category', axis = 1, inplace=True)
data_faspell.rename({'#misspelt':'misspelt'}, axis = 1, inplace=True)

In [4]:
data_faspell.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4858 entries, 0 to 4857
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   misspelt   4858 non-null   object
 1   corrected  4858 non-null   object
dtypes: object(2)
memory usage: 76.0+ KB


In [5]:
data_faspell.head()

,misspelt,corrected
0,آاهي,آگاهي
1,آبات,آیات
2,آبباشد,آب باشد
3,آبد,آید
4,آبری,عابری


#### 1.2.2 context sensitive

This is a real-world test set for errors and context sensitive spelling errors for Persian language. This test set contains 1100 context sensitive errors

In [6]:
data_context = pd.read_csv('data/context_sensitive.txt', header = 0, 
                            names = ['correct', 'misspelt', 'error_cat', 'sentence', 'nan'], 
                            sep = '\t').drop('nan', axis=1)
data_context.head()

,correct,misspelt,error_cat,sentence
0,فرنگي,فرهنگي,Insertion,كاهش قيمت گوجه فرنگي و خيار در ميادين ميوه و ت...
1,فرنگي,فرهنگي,Insertion,خوردن گوجه فرهنگي كه حرام است !
2,فرنگي,فرهنگي,Insertion,گوجه فرهنگي
3,فرنگي,فرهنگي,Insertion,سيب زميني 58000 كيلو ، گوجه فرهنگي 18000 كيلو
4,فرنگي,فرهنگي,Insertion,اجازه فروش كارخانه كشمش ، رب گوجه فرهنگي تاكست...


### 1.3 Combining the datasets

##  2. Preprocess the data

In this step we will ...

In [7]:
# create vocabulary dictionary
# map each persian dictionary to a unique number

vocab = {' ':0 , 'آ':1 , 'ا':2 , 'ب':3 , 'پ':4 , 'ت':5 , 'ث':6 , 'ج':7 , 'چ':8 , 'ح':9 , 'خ':10 ,
 'د':11 , 'ذ':12 , 'ر':13 , 'ز':14 , 'ژ':15 , 'س':16 , 'ش':17 , 'ص':18 , 'ض':19 , 'ط':20 , 'ظ':21 , 
 'ع':22 , 'غ':23 , 'ف':24 , 'ق':25 , 'ک':26 , 'گ':27 , 'ل':28 , 'م':29 , 'ن':30 , 'و':31 , 'ه':32 , 
 'ی':33, '<unk>':34, '<pad>':35}

# create inverse vocabulary dictionary
# map each number to its corresponding index in dictionary

inv_vocab = {0:' ' , 1:'آ' , 2:'ا' , 3:'ب' , 4:'پ' , 5:'ت' , 6:'ث' , 7:'ج' , 8:'چ' , 9:'ح' , 10:'خ' ,
 11:'د' , 12:'ذ' , 13:'ر' , 14:'ز', 15:'ژ' , 16:'س' , 17:'ش' , 18:'ص' , 19:'ض' , 20:'ط' , 21:'ظ' , 
 22:'ع' , 23:'غ' , 24:'ف' , 25:'ق' , 26:'ک' , 27:'گ' , 28:'ل' , 29:'م' , 30:'ن' , 31:'و' , 32:'ه' , 
 33:'ی', 34:'<unk>', 35:'<pad>'}

* We will set T=50
    * We assume T is the maximum length of the query.
    * If we get a longer input, we would have to truncate it.

In [8]:
T = 50
X, Y, Xoh, Yoh = preprocess_data(list(data_faspell.itertuples(index=False, name=None)), vocab, T)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (4858, 50)
Y.shape: (4858, 50)
Xoh.shape: (4858, 50, 36)
Yoh.shape: (4858, 50, 36)


We now have:

* `X`: a processed version of the human queries in the training set.
    - Each character in X is replaced by an index (integer) mapped to the character using `vocab`.
    - Each date is padded to ensure a length of `T` using a special character (< pad >).
    - `X.shape = (m, T)` where m is the number of training examples in a batch.
    
* `Y`: a processed version of the machine readable dates in the training set.
    - Each character is replaced by the index (integer) it is mapped to in `vocab`.
    - `Y.shape = (m, T)`.
* `Xoh`: one-hot version of `X`
    - Each index in X is converted to the one-hot representation (if the index is 2, the one-hot version has the index position 2 set to 1, and the remaining positions are 0.
    - `Xoh.shape = (m, T, len(vocab))`.
* `Yoh`: one-hot version of `Y`
    - Each index in `Y` is converted to the one-hot representation.
    - `Yoh.shape = (m, T, len(vocab))`.


* Let's also look at an example of preprocessed training examples.

In [9]:
index = 0
# print("Source query:", dataset[index][0])
# print("Target query:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])


Source after preprocessing (indices): [ 1  2 32 33 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35
 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35
 35 35]
Target after preprocessing (indices): [ 1 27  2 32 33 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35
 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35
 35 35]

Source after preprocessing (one-hot): [[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Target after preprocessing (one-hot): [[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


## 3. Spelling correction using LSTMs with Attention

* If you had to translate a book's paragraph from French to English, you would not read the whole paragraph, then close the book and translate. 
* Even during the translation process, you would read/re-read and focus on the parts of the French paragraph corresponding to the parts of the English you are writing down. 
* The attention mechanism tells a Neural Machine Translation model where it should pay attention to at any step. 

### 3.1 - Attention Mechanism

In this part, we will implement the attention mechanism presented in the lecture videos. 
* Here is a figure to remind you how the model works. 
    * The diagram on the left shows the attention model. 
    * The diagram on the right shows what one "attention" step does to calculate the attention variables $\alpha^{\langle t, t' \rangle}$.
    * The attention variables $\alpha^{\langle t, t' \rangle}$ are used to compute the context variable $context^{\langle t \rangle}$ for each timestep in the output ($t=1, \ldots, T_y$). 

<table>
<td> 
<img src="images/attn_model.png" style="width:500;height:500px;"> <br>
</td> 
<td> 
<img src="images/attn_mechanism.png" style="width:500;height:500px;"> <br>
</td> 
</table>
<caption><center> **Figure 1**: Neural machine translation with attention</center></caption>

Here are some properties of the model that you may notice: 

#### Pre-attention and Post-attention LSTMs on both sides of the attention mechanism
- There are two separate LSTMs in this model (see diagram on the left): pre-attention and post-attention LSTMs.
- *Pre-attention* Bi-LSTM is the one at the bottom of the picture is a Bi-directional LSTM and comes *before* the attention mechanism.
    - The attention mechanism is shown in the middle of the left-hand diagram.
    - The pre-attention Bi-LSTM goes through $T_x$ time steps
- *Post-attention* LSTM: at the top of the diagram comes *after* the attention mechanism. 
    - The post-attention LSTM goes through $T_y$ time steps. 

- The post-attention LSTM passes the hidden state $s^{\langle t \rangle}$ and cell state $c^{\langle t \rangle}$ from one time step to the next. 

#### An LSTM has both a hidden state and cell state
* In the lecture videos, we were using only a basic RNN for the post-attention sequence model
    * This means that the state captured by the RNN was outputting only the hidden state $s^{\langle t\rangle}$. 
* In this assignment, we are using an LSTM instead of a basic RNN.
    * So the LSTM has both the hidden state $s^{\langle t\rangle}$ and the cell state $c^{\langle t\rangle}$. 

#### Each time step does not use predictions from the previous time step
* The post-attention LSTM at time $t$ does not take the previous time step's prediction $y^{\langle t-1 \rangle}$ as input.
* The post-attention LSTM at time 't' only takes the hidden state $s^{\langle t\rangle}$ and cell state $c^{\langle t\rangle}$ as input. 

#### Concatenation of hidden states from the forward and backward pre-attention LSTMs
- $\overrightarrow{a}^{\langle t \rangle}$: hidden state of the forward-direction, pre-attention LSTM.
- $\overleftarrow{a}^{\langle t \rangle}$: hidden state of the backward-direction, pre-attention LSTM.
- $a^{\langle t \rangle} = [\overrightarrow{a}^{\langle t \rangle}, \overleftarrow{a}^{\langle t \rangle}]$: the concatenation of the activations of both the forward-direction $\overrightarrow{a}^{\langle t \rangle}$ and backward-directions $\overleftarrow{a}^{\langle t \rangle}$ of the pre-attention Bi-LSTM. 

#### Computing "energies" $e^{\langle t, t' \rangle}$ as a function of $s^{\langle t-1 \rangle}$ and $a^{\langle t' \rangle}$
- "e" is called the "energies" variable.
- $s^{\langle t-1 \rangle}$ is the hidden state of the post-attention LSTM
- $a^{\langle t' \rangle}$ is the hidden state of the pre-attention LSTM.
- $s^{\langle t-1 \rangle}$ and $a^{\langle t \rangle}$ are fed into a simple neural network, which learns the function to output $e^{\langle t, t' \rangle}$.
- $e^{\langle t, t' \rangle}$ is then used when computing the attention $a^{\langle t, t' \rangle}$ that $y^{\langle t \rangle}$ should pay to $a^{\langle t' \rangle}$.

- The diagram on the right of figure 1 uses a `RepeatVector` node to copy $s^{\langle t-1 \rangle}$'s value $T_x$ times.
- Then it uses `Concatenation` to concatenate $s^{\langle t-1 \rangle}$ and $a^{\langle t \rangle}$.
- The concatenation of $s^{\langle t-1 \rangle}$ and $a^{\langle t \rangle}$ is fed into a "Dense" layer, which computes $e^{\langle t, t' \rangle}$. 
- $e^{\langle t, t' \rangle}$ is then passed through a softmax to compute $\alpha^{\langle t, t' \rangle}$.
- Note that the diagram doesn't explicitly show variable $e^{\langle t, t' \rangle}$, but $e^{\langle t, t' \rangle}$ is above the Dense layer and below the Softmax layer in the diagram in the right half of figure 1.
- We'll explain how to use `RepeatVector` and `Concatenation` in Keras below. 

#### Implementation Details
   
Let's implement this neural translator. We will start by implementing two functions: `one_step_attention()` and `model()`.

#### one_step_attention
* The inputs to the one_step_attention at time step $t$ are:
    - $[a^{<1>},a^{<2>}, ..., a^{<T_x>}]$: all hidden states of the pre-attention Bi-LSTM.
    - $s^{<t-1>}$: the previous hidden state of the post-attention LSTM 
* one_step_attention computes:
    - $[\alpha^{<t,1>},\alpha^{<t,2>}, ..., \alpha^{<t,T_x>}]$: the attention weights
    - $context^{ \langle t \rangle }$: the context vector:
    
$$context^{<t>} = \sum_{t' = 1}^{T_x} \alpha^{<t,t'>}a^{<t'>}\tag{1}$$ 

##### Clarifying 'context' and 'c'
- In the project, we are calling the context $context^{\langle t \rangle}$.
    - This is to avoid confusion with the post-attention LSTM's internal memory cell variable, which is also denoted $c^{\langle t \rangle}$.